# Libraries

In [61]:
import sys
import PyPDF2
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.embeddings import HuggingFaceEmbeddings, CacheBackedEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.storage import InMemoryStore, LocalFileStore, RedisStore
from langchain.vectorstores import FAISS

# Getting data

In [7]:
!wget 'https://docs.google.com/uc?export=download&id=1f1INWOfJrHTFmbyF_0be5b4u_moz3a4F' -O steve-jobs-commencement.txt

with open('steve-jobs-commencement.txt') as f:
    text = f.read()

text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = text_splitter.split_text(text)

len(texts)

--2023-08-24 15:48:15--  https://docs.google.com/uc?export=download&id=1f1INWOfJrHTFmbyF_0be5b4u_moz3a4F
Resolving docs.google.com (docs.google.com)... 172.217.2.142, 2607:f8b0:4012:805::200e
Connecting to docs.google.com (docs.google.com)|172.217.2.142|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/of2gbngv5d8kcpqepte0o6l909p66bo1/1692892050000/12721472133292131824/*/1f1INWOfJrHTFmbyF_0be5b4u_moz3a4F?e=download&uuid=9cc0e3e5-4e39-4dfb-8412-3995b9a86b79 [following]
--2023-08-24 15:48:15--  https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/of2gbngv5d8kcpqepte0o6l909p66bo1/1692892050000/12721472133292131824/*/1f1INWOfJrHTFmbyF_0be5b4u_moz3a4F?e=download&uuid=9cc0e3e5-4e39-4dfb-8412-3995b9a86b79
Resolving doc-0g-84-docs.googleusercontent.com (doc-0g-84-docs.googleusercontent.com)... 192.178.52.225, 172.217.15.1, 2607:f8b0

Created a chunk of size 310, which is longer than the specified 300
Created a chunk of size 835, which is longer than the specified 300
Created a chunk of size 746, which is longer than the specified 300
Created a chunk of size 422, which is longer than the specified 300
Created a chunk of size 600, which is longer than the specified 300
Created a chunk of size 800, which is longer than the specified 300
Created a chunk of size 325, which is longer than the specified 300
Created a chunk of size 836, which is longer than the specified 300
Created a chunk of size 555, which is longer than the specified 300
Created a chunk of size 310, which is longer than the specified 300
Created a chunk of size 512, which is longer than the specified 300
Created a chunk of size 842, which is longer than the specified 300
Created a chunk of size 460, which is longer than the specified 300
Created a chunk of size 504, which is longer than the specified 300
Created a chunk of size 679, which is longer tha

25

In [22]:
!wget "https://fecolsa.com.co/upload/Actividades%20Equilibrio%20Total/Rowling%2C%20J.%20K.%20-%20%20Harry%20Potter%20y%20la%20piedra%20filosofal.pdf" -O harry1.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--2023-08-24 16:26:00--  https://fecolsa.com.co/upload/Actividades%20Equilibrio%20Total/Rowling%2C%20J.%20K.%20-%20%20Harry%20Potter%20y%20la%20piedra%20filosofal.pdf
Resolving fecolsa.com.co (fecolsa.com.co)... 129.80.167.70
Connecting to fecolsa.com.co (fecolsa.com.co)|129.80.167.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2484031 (2.4M) [application/pdf]
Saving to: ‘harry1.txt’

harry1.txt          100%[===================>]   2.37M  4.86MB/s    in 0.5s    

2023-08-24 16:26:00 (4.86 MB/s) - ‘harry1.txt’ saved [2484031/2484031]



In [30]:
loader = PyPDFLoader("harry1.txt")
pages = loader.load_and_split()

# Getting Hugginface embeddings

In [12]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    cache_folder = "./models"
)

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Faiss-DB

## Creating it

In [16]:
faiss_path= 'data/faiss-db'
plain_faiss = FAISS.from_texts(texts=texts,embedding=hf)
plain_faiss.save_local(faiss_path)
reload_faiss = FAISS.load_local(faiss_path,embeddings=hf)

In [21]:
plain_faiss.index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7f4a9e51e2b0> >

In [31]:
plain_faiss2 = FAISS.from_documents(documents=pages,embedding=hf)

In [32]:
faiss_path2= 'data/faiss-db2'
plain_faiss2.save_local(faiss_path2)

In [37]:
plain_faiss3 = plain_faiss.aadd_documents(documents= pages)

In [39]:
plain_faiss.add_documents(documents= pages)

['d2ea6236-3517-438a-bf2d-a89a6f6e3fa5',
 'ee2b264e-983f-4a39-be92-2dca9da5b70a',
 '2ca95621-9374-4f57-ab74-654caa12f737',
 '3631d920-9562-4172-be4a-3ac9001ad6d7',
 'ba768a6f-e42e-4bc6-aff0-720f1f615373',
 'ab4969b5-eae6-44a4-8a7f-304812f94f75',
 'a54475ae-4bdf-4603-947d-4fcd17915176',
 '9b6abef0-f714-44e2-bd45-7d3681b8349d',
 '2c6c875d-16f2-48c7-a460-2fced8376087',
 '143cd26d-15d7-49c8-a955-747b57a71b9a',
 '44b72a48-582f-4da9-b49d-3cd678916950',
 '87cbc6d3-6947-40d7-9c77-338dd1f50343',
 'a11f2e26-0f8b-4f7d-87b8-d852cf1d1d72',
 '5810dfd5-c222-483f-9345-7850c2ac517b',
 '8b025b34-9b0e-44a8-95a7-9ef223db92f3',
 'a9f30ef1-f3d5-414a-95e5-5557bddeae51',
 '17acaca8-170b-4463-b46b-ce4edc24f699',
 'a9bbae78-f7a8-41d1-b2be-a3708fbf749d',
 '91051ea0-2f63-4a90-8d68-5eafdab5194f',
 '0dade026-bf4a-45f0-aa97-5aaad8b4bb6e',
 'c102370f-0339-45b1-9304-33f1d7af0c3a',
 'b56c701f-2dff-4836-921e-958628ce0020',
 '19193956-e507-433f-b1cd-bfe71b0434b3',
 '8dc06a2e-f909-49dc-9f96-24dd75ad0f77',
 '4a75b734-68a6-

In [40]:
faiss_path3= 'data/faiss-db3'
plain_faiss.save_local(faiss_path3)

In [42]:
size = sys.getsizeof(plain_faiss)

In [43]:
size

56

In [49]:
size = sys.getsizeof(loader)
size

56

In [45]:
size = sys.getsizeof(plain_faiss2)
size

56

## other way

In [51]:
hf.model_name

'sentence-transformers/all-mpnet-base-v2'

In [54]:
fs = LocalFileStore("./cache/")

In [56]:
hf.model_name

'sentence-transformers/all-mpnet-base-v2'

In [58]:
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    hf, fs, namespace=hf.model_name
)

In [59]:
list(fs.yield_keys())

[]

In [64]:
db = FAISS.from_texts(texts, cached_embedder)

In [66]:
db2 = FAISS.from_texts(texts, cached_embedder)

In [65]:
list(fs.yield_keys())[:5]

['sentence-transformers/all-mpnet-base-v2091da8f9-4640-5fe7-b6a1-d762c5ffefc2',
 'sentence-transformers/all-mpnet-base-v232e3d6a7-8b79-5ee0-af34-72aba054ca63',
 'sentence-transformers/all-mpnet-base-v2edf4a095-7fd9-5f80-b8cd-245a5de83915',
 'sentence-transformers/all-mpnet-base-v25b2c8f10-c639-5a98-b4be-6e9660b1a7ca',
 'sentence-transformers/all-mpnet-base-v217f2935e-c21c-5441-bd5d-771a14e8ebaf']